<a href="https://colab.research.google.com/github/Ananyaa-Perumalsamy/Ananyaa-Perumalsamy/blob/main/PS_1_docu3C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install scikit-learn
!pip install pdfplumber
!pip install PyPDF2

  Using cached pdfplumber-0.11.5-py3-none-any.whl.metadata (42 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.7 MB/s eta 0:00:00


In [2]:
from google.colab import files
uploaded = files.upload()

pdf_path = list(uploaded.keys())[0]


Saving Amicus Brief on Behalf of Mississippi, Alabama, Alaska, Arkansas etc....pdf to Amicus Brief on Behalf of Mississippi, Alabama, Alaska, Arkansas etc....pdf


In [8]:
import pdfplumber

def extract_text(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = ''
        for page in pdf.pages:
            text += page.extract_text()
    return text

document_text = extract_text(pdf_path)


In [10]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [11]:
import spacy
from nltk.corpus import stopwords
import string

nlp = spacy.load('en_core_web_sm')
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if token.text not in stop_words and token.text not in string.punctuation]
    return ' '.join(tokens)

preprocessed_text = preprocess_text(document_text)


/usr/local/lib/python3.11/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform([preprocessed_text])


In [16]:
!pip install --upgrade pip
!pip install pdfplumber transformers spacy scikit-learn datasets nltk
!python -m spacy download en_core_web_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 29.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [20]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

In [26]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_function(text):
    return tokenizer(text, padding="max_length", truncation=True, return_tensors="pt")

# Tokenize the text
inputs = tokenize_function(preprocessed_text)

# Add dummy labels
labels = torch.tensor([1] * inputs['input_ids'].shape[0])

# Convert inputs to Dataset format
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

dataset = CustomDataset(inputs, labels)

# Fine-tune the model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="no",  # Set evaluation strategy to 'no'
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to="none"  # Disable wandb logging
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss


TrainOutput(global_step=3, training_loss=0.4872499306996663, metrics={'train_runtime': 36.9527, 'train_samples_per_second': 0.081, 'train_steps_per_second': 0.081, 'total_flos': 789333166080.0, 'train_loss': 0.4872499306996663, 'epoch': 3.0})

In [28]:
inputs = tokenizer(preprocessed_text, return_tensors="pt", padding=True, truncation=True)
outputs = model(**inputs)
logits = outputs.logits

# Get the number of available predictions
num_predictions = logits.size(1)

# Request top k items based on the available number of predictions
predicted_class = torch.topk(logits, min(10, num_predictions)).indices.tolist()


In [29]:
print(document_text)

Case 2:22-cv-00223-Z Document 100 Filed 02/13/23 Page 1 of 26 PageID 3760
UNITED STATES DISTRICT COURT
NORTHERN DISTRICT OF TEXAS
AMARILLO DIVISION
ALLIANCE FOR HIPPOCRATIC
MEDICINE, et al.,
Plaintiffs,
v. Case No. 2:22-cv-00223-Z
U.S. FOOD AND DRUG
ADMINISTRATION, et al.,
Defendants.
AMICUS CURIAE BRIEF OF MISSISSIPPI, ALABAMA, ALASKA,
ARKANSAS, FLORIDA, GEORGIA, IDAHO, INDIANA, IOWA, KANSAS,
KENTUCKY, LOUISIANA, MONTANA, NEBRASKA, OHIO, OKLAHOMA,
SOUTH CAROLINA, SOUTH DAKOTA, TENNESSEE, TEXAS, UTAH, AND
WYOMING IN SUPPORT OF PLAINTIFFS’ MOTION
FOR PRELIMINARY INJUNCTIONCase 2:22-cv-00223-Z Document 100 Filed 02/13/23 Page 2 of 26 PageID 3761
TABLE OF CONTENTS
Page
TABLE OF AUTHORITIES .......................................................................................... ii
INTRODUCTION, INTEREST OF AMICI CURIAE,
AND SUMMARY OF ARGUMENT .............................................................................. 1
BACKGROUND ......................................................

In [30]:
print(preprocessed_text)


case 2:22 cv-00223 Z Document 100 file 02/13/23 Page 1 26 PageID 3760 
 UNITED STATES DISTRICT COURT 
 NORTHERN DISTRICT of TEXAS 
 AMARILLO DIVISION 
 ALLIANCE for HIPPOCRATIC 
 MEDICINE et al 
 plaintiff 
 v. Case no 2:22 cv-00223 Z 
 U.S. FOOD and DRUG 
 ADMINISTRATION et al 
 defendant 
 AMICUS CURIAE BRIEF of MISSISSIPPI ALABAMA ALASKA 
 ARKANSAS FLORIDA GEORGIA IDAHO INDIANA IOWA KANSAS 
 KENTUCKY LOUISIANA MONTANA NEBRASKA OHIO OKLAHOMA 
 SOUTH CAROLINA south DAKOTA TENNESSEE TEXAS UTAH and 
 WYOMING in SUPPORT of PLAINTIFFS ' MOTION 
 for PRELIMINARY INJUNCTIONCase 2:22 cv-00223 Z Document 100 file 02/13/23 Page 2 26 PageID 3761 
 table of content 
 Page 
 table of AUTHORITIES .......................................................................................... ii 
 INTRODUCTION interest of AMICI CURIAE 
 and SUMMARY of ARGUMENT .............................................................................. 1 
 BACKGROUND ....................................................

In [31]:
print(inputs)

{'input_ids': tensor([[  101,  2553,  1016,  1024,  2570, 26226,  1011,  4002, 19317,  2509,
          1062,  6254,  2531,  5371,  6185,  1013,  2410,  1013,  2603,  3931,
          1015,  2656,  3931,  3593,  4261, 16086,  2142,  2163,  2212,  2457,
          2642,  2212,  1997,  3146, 23204, 10486,  2407,  4707,  2005,  5099,
          6873, 17510,  4200,  3802,  2632, 20579,  1058,  1012,  2553,  2053,
          1016,  1024,  2570, 26226,  1011,  4002, 19317,  2509,  1062,  1057,
          1012,  1055,  1012,  2833,  1998,  4319,  3447,  3802,  2632, 13474,
         26445,  7874, 12731,  4360,  2063,  4766,  1997,  5900,  6041,  7397,
          6751,  3516,  4108,  9795,  5242,  5947,  5111,  5612,  5773,  8124,
          8506,  4058,  5858,  2148,  3792,  2148,  7734,  5298,  3146,  6646,
          1998, 10622,  1999,  2490,  1997, 23953,  1005,  4367,  2005,  8824,
         22928, 18382,  1016,  1024,  2570, 26226,  1011,  4002, 19317,  2509,
          1062,  6254,  2531,  5371,  

In [32]:
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    print(logits)

tensor([[-0.1392,  0.4932]])


In [33]:
num_predictions = logits.size(1)
top_k = min(10, num_predictions)
predicted_class = torch.topk(logits, top_k).indices.tolist()
print(predicted_class)

[[1, 0]]


In [34]:
import pandas as pd
from IPython.display import display

# Assuming `key_items` is the list of key items obtained from your model
key_items = [
    "Key Item 1: Argument summary 1",
    "Key Item 2: Argument summary 2",
    "Key Item 3: Argument summary 3",
    # Add more key items as needed
]

# Create a DataFrame to display the key items
df = pd.DataFrame(key_items, columns=["Key Items"])
display(df)


,Key Items
0,Key Item 1: Argument summary 1
1,Key Item 2: Argument summary 2
2,Key Item 3: Argument summary 3


from matplotlib import pyplot as plt
import seaborn as sns
df.groupby('Key Items').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)